In [1]:
!pip3 install beautifulsoup4
!pip install Selenium
!pip install webdriver-manager
!pip install html_table_parser
!pip install chromedriver-autoinstaller
!pip install --upgrade beautifulsoup4
!pip install --upgrade html5lib

  Using cached beautifulsoup4-4.4.1-py3-none-any.whl (81 kB)
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.10.0
    Uninstalling beautifulsoup4-4.10.0:
      Successfully uninstalled beautifulsoup4-4.10.0
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.4.1
    Uninstalling beautifulsoup4-4.4.1:
      Successfully uninstalled beautifulsoup4-4.4.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
html-table-parser 0.1.0 requires beautifulsoup4==4.4.1, but you have beautifulsoup4 4.10.0 which is incompatible.


In [2]:
from selenium import webdriver 
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import requests
from html_table_parser import parser_functions as parser
from urllib.request import urlopen
from time import sleep


import chromedriver_autoinstaller


## 드라이버 가져오기

In [22]:
driver = webdriver.Chrome(ChromeDriverManager().install())
sleep(5)
url = 'https://bd.kma.go.kr/kma2020/fs/energySelect1.do?pageNum=5&menuCd=F050701000'
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\hongr\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


## 크롤링 항목

* 오늘(날짜)
* 각 시간에 대한 정보, 누적발전량, 일사량
* 전국에 대한 1차 수집

In [4]:
today = driver.find_element_by_id("thToday").text
print(today)

오늘(10/11)


In [5]:
area_name = driver.find_element_by_id("areaName").text
print(area_name)

전국


In [6]:
element = driver.find_element_by_xpath('//*[@id="toEnergy"]/tr[1]').text.split()
sleep(5)
print("시간당 출력되는 정보:",element)

시간당 출력되는 정보: ['0시', '0.00', '0.00', '0.0', '20.6', '0.8', '0.00', '0.00', '0.0', '15.1', '5.1']


In [7]:
time = []
elect = []
sun = []
for i in range(1,24):
    
    tr = str(i)
    string = driver.find_element_by_xpath(f'//*[@id="toEnergy"]/tr[{i}]').text.split()
    time.append(string[0])
    elect.append(string[2])
    sun.append(string[3])

print('time : ',time)
print('elect : ',elect)
print('sun : ',sun)

time :  ['0시', '1시', '2시', '3시', '4시', '5시', '6시', '7시', '8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시', '21시', '22시']
elect :  ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.05', '103.27', '339.55', '708.36', '1187.45', '1690.82', '2171.05', '2576.30', '2901.44', '3140.22', '3276.79', '3317.47', '3317.47', '3317.47', '3317.47', '3317.47']
sun :  ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '1.7', '10.3', '41.9', '51.5', '59.0', '52.3', '62.4', '74.7', '77.3', '83.0', '45.3', '10.7', '0.0', '0.0', '0.0', '0.0']


In [8]:
my_dict = {"누적발전량":elect,"일사량":sun}
my_df = pd.DataFrame(my_dict,index=time).rename_axis(f"전국:{today}")
display(my_df)

,누적발전량,일사량
전국:오늘(10/11),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.05,1.7
8시,103.27,10.3


## 각 지역별로 시간대별 누적발전량, 일사량 정보를 수집

*  서울특별시
*  경기도
*  인천광역시
*  대전광역시
*  세종특별자치시
*  광주광역시
*  울산광역시
*  부산광역시
*  제주특별자치도
*  강원도
*  충청북도
*  충청남도
*  경상북도
*  경상남도
*  전라북도
*  전라남도


## 각 지역별로 크롤링 전, 리스트에 담고, 반복문을 돌아 크롤링.

### 1. 셀레니움을 이용해 각 지역을 클릭

* 필요한 변수 : 오늘날짜를 담을 변수, 

### 2. 각 지역을 클릭시 생성된 DataFrame을 위에서 구축한 함수로 변수에 저장

 
### 3. 출력

* today = driver.find_element_by_id("thToday").text       == 오늘 정보
* area_name = driver.find_element_by_id("areaName").text  == 지역 이름
* driver.find_element_by_xpath('//*[@id="info_4200000000"]/span').click()    == 지역 클릭

In [23]:
서울특별시     = '//*[@id="info_1100000000"]/span' 
경기도         = '//*[@id="info_4100000000"]/span' 
인천광역시     = '//*[@id="info_2800000000"]/span'
대전광역시     = '//*[@id="info_3000000000"]/span'
세종특별자치시 = '//*[@id="info_3600000000"]/span'  
광주광역시     = '//*[@id="info_2900000000"]/span'
대구광역시     = '//*[@id="info_2700000000"]/span'
울산광역시     = '//*[@id="info_3100000000"]/span'
부산광역시     = '//*[@id="info_2600000000"]/span'
제주특별자치도 = '//*[@id="info_5000000000"]/span'
강원도         = '//*[@id="info_4200000000"]/span' 
충청북도       = '//*[@id="info_4300000000"]/span'
충청남도       = '//*[@id="info_4400000000"]/span'
경상북도       = '//*[@id="info_4700000000"]/span'
경상남도       = '//*[@id="info_4800000000"]/span'
전라북도       = '//*[@id="info_4500000000"]/span'
전라남도       = '//*[@id="info_4600000000"]/span'

In [19]:
place = [서울특별시   
,경기도     
,인천광역시   
,대전광역시   
,세종특별자치시 
,광주광역시
,대구광역시         
,울산광역시   
,부산광역시   
,제주특별자치도 
,강원도     
,충청북도    
,충청남도    
,경상북도    
,경상남도    
,전라북도    
,전라남도    ]

### 방식 변경

*def inforamtion(지역) ; 지역에 대한 dataframe 반환 변수* 에서

* def inforamtion_by_selector(지역) 로 **변경**


In [19]:
# def information(area):
    
#     button = driver.find_element_by_xpath(area)
#     driver.execute_script("arguments[0].click();", button)
    
        
#     time = []
#     elect = []
#     sun = []
#     for i in range(1,24):

#         string = driver.find_element_by_xpath(f'//*[@id="toEnergy"]/tr[{i}]').text.split()
#         time.append(string[0])
#         elect.append(string[2])
#         sun.append(string[3])
    
    
#     today = driver.find_element_by_id("thToday").text       #오늘 정보
#     area_name = driver.find_element_by_id("areaName").text  #지역 이름

#     my_dict = {"누적발전량":elect,"일사량":sun}
#     area_my_df = pd.DataFrame(my_dict,index=time).rename_axis(f"{area_name}:{today}")
    
#     return area_my_df

### 데이터 프레임이 한번에 바뀌지 않는 현상

* xpath -> selector 로 메소드 변경
* dataframe 초기화

다양한 방법을 시도해봤지만, 한 실행라인을 두번반복해야 제대로 출력이 됨.

In [35]:
def information_by_selector(area):
    
    button = driver.find_element_by_xpath(area)
    driver.execute_script("arguments[0].click();", button)
    
    col_names =  ['누적발전량','일사량']
    area_my_df  = pd.DataFrame(columns = col_names)
    area_my_df = area_my_df.drop(columns=['누적발전량','일사량'], axis=1,inplace=True)
    
        
    time = []
    elect = []
    sun = []
    
    for i in range(1,24):

        
        everytime = driver.find_elements_by_css_selector(f'#toEnergy > tr:nth-child({i})')

        one_line = everytime[0].text.split()
        time.append(one_line[0])
        elect.append(one_line[2])
        sun.append(one_line[3])
    
    
    today = driver.find_element_by_id("thToday").text       #오늘 정보
    area_name = driver.find_element_by_id("areaName").text  #지역 이름

    my_dict = {"누적발전량":elect,"일사량":sun}
    area_my_df = pd.DataFrame(my_dict,index=time).rename_axis(f"{area_name}:{today}")
    
    return area_my_df

## 각 지역에 대한 정보를 변수에 저장

In [25]:
# 서울 = information_by_selector(서울특별시)
# 경기 = information_by_selector(경기도)
# 인천 = information_by_selector(인천광역시)
# 대전 = information_by_selector(대전광역시)
# 세종 = information_by_selector(세종특별자치시)
# 광주 = information_by_selector(광주광역시)
# 대구 = information_by_selector(대구광역시)
# 울산 = information_by_selector(울산광역시)
# 부산 = information_by_selector(부산광역시)
# 제주 = information_by_selector(제주특별자치도)
# 강원 = information_by_selector(강원도)
# 충북 = information_by_selector(충청북도)
# 충남 = information_by_selector(충청남도)
# 경북 = information_by_selector(경상북도)
# 경남 = information_by_selector(경상남도)
# 전북 = information_by_selector(전라북도)
# 전남 = information_by_selector(전라남도)

In [27]:
서울 = information_by_selector(서울특별시)
서울

,누적발전량,일사량
서울특별시:오늘(10/11),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,0.8
8시,0.23,14.5


In [29]:
경기 = information_by_selector(경기도)
경기

,누적발전량,일사량
경기도:오늘(10/11),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,0.6
8시,2.21,12.5


In [36]:
인천 = information_by_selector(인천광역시)
인천

,누적발전량,일사량
인천광역시:오늘(10/11),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,1.0
8시,0.49,21.7


In [38]:
대전 = information_by_selector(대전광역시)
대전

,누적발전량,일사량
대전광역시:오늘(10/11),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,0.7
8시,0.19,12.0


In [40]:
세종 = information_by_selector(세종특별자치시)
세종

,누적발전량,일사량
세종특별자치시:오늘(10/11),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,0.5
8시,0.15,10.5


In [32]:
광주 = information_by_selector(광주광역시)
display(광주)

,누적발전량,일사량
광주광역시:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,6.5
8시,5.77,129.8


In [30]:
대구 = information_by_selector(대구광역시)
대구

,누적발전량,일사량
대구광역시:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,12.7
8시,3.83,124.4


In [34]:
울산 = information_by_selector(울산광역시)
울산

,누적발전량,일사량
울산광역시:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.48,24.2
8시,3.48,180.1


In [36]:
부산 = information_by_selector(부산광역시)
부산

,누적발전량,일사량
부산광역시:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.46,17.3
8시,10.59,134.0


In [39]:
제주 = information_by_selector(제주특별자치도)
제주


,누적발전량,일사량
제주특별자치도:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,2.7
8시,10.37,57.6


In [41]:
강원 = information_by_selector(강원도)
강원

,누적발전량,일사량
강원도:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,18.9
8시,20.33,152.9


In [25]:
충북 = information_by_selector(충청북도)
충북

,누적발전량,일사량
충청북도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,5.0
8시,8.96,81.2


In [43]:
충남 = information_by_selector(충청남도)
충남

,누적발전량,일사량
충청남도:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,7.0
8시,50.83,123.4


In [45]:
경북 = information_by_selector(경상북도)
경북

,누적발전량,일사량
경상북도:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.76,11.2
8시,43.34,135.6


In [47]:
경남 = information_by_selector(경상남도)
경남

,누적발전량,일사량
경상남도:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.03,13.6
8시,27.64,165.2


In [29]:
전북 = information_by_selector(전라북도)
전북

,누적발전량,일사량
전라북도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.15,5.1
8시,13.43,85.0


In [23]:
전남 = information_by_selector(전라남도)
전남

,누적발전량,일사량
전라남도:오늘(10/10),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,5.6
8시,22.58,122.0
